In [1]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F

import pandas as pd
import seaborn as sns
import numpy as np
import h5py

import matplotlib.pyplot as plt
from collections import OrderedDict

sns.set_style("white")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random_seed = 1
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

In [3]:
# The reynolds numbers from 10 to 90 are used for training 
# The model is evaluated on the last 10 reynolds numbers from 91 to 100

outputs = np.zeros((90-10+1, 2223))
for Re in range(10,91,1):
    outputs[Re-10] = np.loadtxt('pinns/pretrained/weights/weights_'+str(Re)+'.txt')
    
# Inputs represent the reynolds numbers used for training
# Outputs are the outputs of the hypernetwork which are the weights of the PINN
inputs = np.arange(10,91,1)
inputs = np.reshape(inputs, (inputs.shape[0],1))
inputs = Variable(torch.from_numpy(inputs).float(), requires_grad=False).to(device)
outputs = Variable(torch.from_numpy(outputs).float(), requires_grad=False).to(device)
#print("inputs", inputs.shape)
#print("outputs", outputs.shape)

In [4]:
class HyperNetwork(nn.Module):
    def __init__(self):
        super(HyperNetwork, self).__init__()
        
        self.layers = nn.Sequential(nn.Linear(1,512),
                      nn.ReLU(),
                      nn.Linear(512,512),
                      nn.ReLU(),
                      nn.Linear(512,256),
                      nn.ReLU(),
                      nn.Linear(256,256),
                      nn.ReLU(),
                      nn.Linear(256,128),
                      nn.ReLU(),
                      nn.Linear(128,2223))    
        
    def forward(self, Re):        
        weights = self.layers(Re)
        return weights

In [5]:
hnet = HyperNetwork()
hnet = hnet.to(device)
mse_cost_function = nn.MSELoss() 
optimizer = torch.optim.Adam(hnet.parameters(), lr=1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1, last_epoch=-1, verbose=False)

In [ ]:
iterations = 50000
for epoch in range(iterations):
    optimizer.zero_grad()
    
    hnet_out = hnet.forward(inputs)
    Loss = mse_cost_function(hnet_out, outputs)
    Loss.backward()
    optimizer.step()
    scheduler.step()

    with torch.autograd.no_grad():
        if epoch%1000 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print('Epoch %d, LR: %.4e, Loss: %.4e' % (epoch, current_lr, Loss))

Epoch 0, LR: 1.0000e-03, Loss: 3.4437e-01
Epoch 1000, LR: 1.0000e-03, Loss: 1.1520e-01
Epoch 2000, LR: 1.0000e-03, Loss: 1.1147e-01
Epoch 3000, LR: 1.0000e-03, Loss: 9.6513e-02
Epoch 4000, LR: 1.0000e-03, Loss: 1.0591e-01
Epoch 5000, LR: 1.0000e-03, Loss: 9.4622e-02


In [ ]:
# outputs = np.zeros((10, 2223))
# for Re in range(91,101,1):
#     outputs[Re-91] = np.loadtxt('pinns/pretrained/weights/weights_'+str(Re)+'.txt')
    
# inputs = np.arange(91,101,1)
# inputs = np.reshape(inputs, (inputs.shape[0],1))
# inputs = Variable(torch.from_numpy(inputs).float(), requires_grad=False).to(device)
# outputs = Variable(torch.from_numpy(outputs).float(), requires_grad=False).to(device)
# pred_outputs = hnet.forward(inputs)
# L2_error = torch.linalg.norm(outputs-pred_outputs,2)/torch.linalg.norm(outputs,2)
# print('Relative L2 Error: %e \n' % (L2_error))

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
    
    def forward(self, weights, x):
        h = F.tanh(F.linear(x, weight = weights[0, :40].view(20, 2), bias = weights[0, 40:60].view(20)))
        i = 60
        for _ in range(5):
            h = F.tanh(F.linear(h, weight = weights[0, i:i+400].view(20, 20), bias = weights[0, i+400:i+420].view(20)))
            i += 420
        h = F.linear(h, weight = weights[0, i:i+60].view(3, 20), bias = weights[0, i+20:i+23].view(3))
        return h

In [ ]:
pinn = PINN()

f = h5py.File('data/colocation_points.h5', 'r')
X_col = f['X_col']
Y_col = f['Y_col']
X_col = np.asarray(X_col)
Y_col = np.asarray(Y_col)
X_col = Variable(torch.from_numpy(X_col).float(), requires_grad=True).to(device)
Y_col = Variable(torch.from_numpy(Y_col).float(), requires_grad=True).to(device)
inputs = torch.cat([X_col, Y_col], axis=1)
f.close()

for Re in range(91,101,1):
    Re_tensor = torch.tensor(Re, dtype=torch.float32).view(1,1)
    predicted_weights = hnet.forward(Re_tensor)
    predicted_solution = pinn.forward(predicted_weights, inputs)
    
    f = h5py.File('data/Analytical_Solutions/colocation/Re_{}.h5'.format(Re), 'r')
    S_col = f['S_col']
    S_col = np.asarray(S_col)
    analytical_solution = Variable(torch.from_numpy(S_col).float(), requires_grad=True).to(device)
    f.close()
    
    print("Re: ", Re)
    # Mean L2 Error
    error = torch.linalg.norm(analytical_solution-predicted_solution,2)
    print('Mean L2 Error: %.4e \n' % (error))
    # Relative L2 Error
    L2_error = torch.linalg.norm(analytical_solution-predicted_solution,2)/torch.linalg.norm(analytical_solution,2)
    print('Relative L2 Error: %.4e \n' % (L2_error))